# Project: **Gauss-Newton Method** in **DFO-GN: Derivative-Free Nonlinear Least-Squares Solver**

## About the method

Initially, we've used Newton's method as a rootfinding algorithm. However in the regression activity, we've learned that Newton's method can also be used for solving the optimization problem. In exploring non-linear regression, the Gauss-Newton method, a modification of Newton's method, is used to solve the nonlinear least squares problem, where it minimizes the sum of squared function values. The Gauss-Newton method is preferred over Newton's method because second derivatives that are challenging to compute are not required.

## About the software

Repository: https://github.com/numericalalgorithmsgroup/dfogn

This software package implements the classical Gauss-Newton method as a derivative-free optimization algorithm designed to solve the nonlinear least squares problem:

\begin{split}\min_{x\in\mathbb{R}^n}  &\quad  f(x) := \sum_{i=1}^{m}r_{i}(x)^2 \\
\text{s.t.} &\quad  a \leq x \leq b\end{split}

This software was developed by Lindon Roberts and Carolina Cartis for their [research paper](https://link.springer.com/article/10.1007%2Fs12532-019-00161-7). The majority of methods that solve the nonlinear least squares problem involves a derivative-based algorithm. However, this derivative-free algorithm (DFO-GN) is preferred over a derivative-based algorithm if the residuals are noisy or if the residuals are expensive to evaluate. The software is developed in Python, is combined with SciPy and NumPy, and can be easily installed as:

In [35]:
! pip install dfogn

## Method as it appears in the software

The main usage of DFO-GN is to use the following function *solve*:



The input *objfun* is a Python function which takes an input 

$$x\in\mathbb{R}^n$$

where the input is a one-dimensional NumPy array. The input *x0* is the starting point for the solver and should be used as the best estimate to the solution

$$x_{min}\in\mathbb{R}^n$$

## Examples and Usage

A common test problem is to minimize the [Rosenbrock function](https://numericalalgorithmsgroup.github.io/dfogn/build/html/userguide.html#how-to-use-dfo-gn), written as a least-squares problem and using common starting point $x_0=(-1.2,1)$.

\begin{split}\min_{(x_1,x_2)\in\mathbb{R}^2}  &\quad  [10(x_2-x_1^2)]^2 + [1-x_1]^2 \\\end{split}



In [8]:
# DFO-GN example: minimize the Rosenbrock function
from __future__ import print_function
import numpy as np
import dfogn

# Define the objective function
def rosenbrock(x):
    return np.array([10.0 * (x[1] - x[0] ** 2), 1.0 - x[0]])

# Define the starting point
x0 = np.array([-1.2, 1.0])

# Call DFO-GN
soln = dfogn.solve(rosenbrock, x0)

# Display output
print(" *** DFO-GN results *** ")
print("Solution xmin = %s" % str(soln.x))
print("Objective value f(xmin) = %.10g" % soln.f)
print("Needed %g objective evaluations" % soln.nf)
print("Residual vector = %s" % str(soln.resid))
print("Approximate Jacobian = %s" % str(soln.jacobian))
print("Exit flag = %g" % soln.flag)
print(soln.msg)

 *** DFO-GN results *** 
Solution xmin = [1. 1.]
Objective value f(xmin) = 1.232595164e-30
Needed 33 objective evaluations
Residual vector = [-1.11022302e-15  0.00000000e+00]
Approximate Jacobian = [[-2.00180000e+01  1.00000000e+01]
 [-1.00000000e+00  5.44052672e-15]]
Exit flag = 0
Success: Objective is sufficiently small


As mentioned, DFO-GN is particularly effective when the residual (*objfun*) is noisy. Specifically, DFO-GN is preferred over derivative-based solvers such as SciPy. Here's an example:

In [9]:
# DFO-GN example: minimize the Rosenbrock function
from __future__ import print_function
import numpy as np
import dfogn

# Define the objective function
def rosenbrock(x):
    return np.array([10.0 * (x[1] - x[0] ** 2), 1.0 - x[0]])

# Modified objective function: add 1% Gaussian noise
def rosenbrock_noisy(x):
    return rosenbrock(x) * (1.0 + 1e-2 * np.random.normal(size=(2,)))

# Define the starting point
x0 = np.array([-1.2, 1.0])

# Set random seed (for reproducibility)
np.random.seed(0)

print("Demonstrate noise in function evaluation:")
for i in range(5):
    print("objfun(x0) = %s" % str(rosenbrock_noisy(x0)))
print("")

# Call DFO-GN
soln = dfogn.solve(rosenbrock_noisy, x0)

# Display output
print(" *** DFO-GN results *** ")
print("Solution xmin = %s" % str(soln.x))
print("Objective value f(xmin) = %.10g" % soln.f)
print("Needed %g objective evaluations" % soln.nf)
print("Residual vector = %s" % str(soln.resid))
print("Approximate Jacobian = %s" % str(soln.jacobian))
print("Exit flag = %g" % soln.flag)
print(soln.msg)

# Compare with a derivative-based solver
import scipy.optimize as opt
soln = opt.least_squares(rosenbrock_noisy, x0)

print("")
print(" *** SciPy results *** ")
print("Solution xmin = %s" % str(soln.x))
print("Objective value f(xmin) = %.10g" % (2.0 * soln.cost))
print("Needed %g objective evaluations" % soln.nfev)
print("Exit flag = %g" % soln.status)
print(soln.message)

Demonstrate noise in function evaluation:
objfun(x0) = [-4.4776183   2.20880346]
objfun(x0) = [-4.44306447  2.24929965]
objfun(x0) = [-4.48217255  2.17849989]
objfun(x0) = [-4.44180389  2.19667014]
objfun(x0) = [-4.39545837  2.20903317]

 *** DFO-GN results *** 
Solution xmin = [0.99999987 0.99999964]
Objective value f(xmin) = 7.637183294e-13
Needed 30 objective evaluations
Residual vector = [-8.63477757e-07  1.34627233e-07]
Approximate Jacobian = [[-1.96795283e+01  9.91949566e+00]
 [-1.00447169e+00  5.59078195e-04]]
Exit flag = 0
Success: Objective is sufficiently small

 *** SciPy results *** 
Solution xmin = [-1.19999887  1.00000341]
Objective value f(xmin) = 23.80789053
Needed 6 objective evaluations
Exit flag = 3
`xtol` termination condition is satisfied.


The following example demostrates why DFO-GN is preferred over derivative-based solvers such as SciPy shown by the results above. SciPy's derivative based solver is unable to make any progress when the residual is noisy while DFO-GN can.

Another example of DFO-GN's usage is solving a nonlinear system of equations. From [here](https://support.sas.com/documentation/cdl/en/imlug/66112/HTML/default/viewer.htm#imlug_genstatexpls_sect004.htm), the following set of equations are: 

\begin{split}x_1 + x_2 - x_1 x_2 + 2 &= 0, \\
x_1 \exp(-x_2) - 1 &= 0.\end{split}

In [11]:
# DFO-GN example: Solving a nonlinear system of equations
# http://support.sas.com/documentation/cdl/en/imlug/66112/HTML/default/viewer.htm#imlug_genstatexpls_sect004.htm

from __future__ import print_function
import math
import numpy as np
import dfogn

# Want to solve:
#   x1 + x2 - x1*x2 + 2 = 0
#   x1 * exp(-x2) - 1   = 0
def nonlinear_system(x):
    return np.array([x[0] + x[1] - x[0]*x[1] + 2,
                     x[0] * math.exp(-x[1]) - 1.0])

# Warning: if there are multiple solutions, which one
#          DFO-GN returns will likely depend on x0!
x0 = np.array([0.1, -2.0])

soln = dfogn.solve(nonlinear_system, x0)

# Display output
print(" *** DFO-GN results *** ")
print("Solution xmin = %s" % str(soln.x))
print("Objective value f(xmin) = %.10g" % soln.f)
print("Needed %g objective evaluations" % soln.nf)
print("Residual vector = %s" % str(soln.resid))
print("Exit flag = %g" % soln.flag)
print(soln.msg)

 *** DFO-GN results *** 
Solution xmin = [ 0.09777309 -2.32510588]
Objective value f(xmin) = 2.916172822e-16
Needed 13 objective evaluations
Residual vector = [-1.38601752e-09 -1.70204653e-08]
Exit flag = 0
Success: Objective is sufficiently small


In this example, the entries of the residual vector are very small indicating that the two equations are solved to a high accuracy.

### Open questions

* The authors of this software suggest that derivative-free optimization algorithms (DFO-GN) are preferred over derivative-based algorithms only in a few situations, otherwise use the derivative-based algorithms. Then, derivative-free optimization algorithms fills a convenience that derivative-based algorithms fail to solve. Is this a fair statement? Will we see more potential use case for the derivative-free optimization algorithms?